In [36]:
import torch

n, p = 10, 3
col = 0

In [53]:
X = torch.randn([n, p]) * 0.5
X = torch.softmax(X, 0)
B = torch.matmul(X.T, X)
B = torch.inverse(B)

In [54]:
new_col = v = torch.softmax(torch.randn([n, 1]) * 0.5, 0)

In [56]:
u1 = torch.matmul(X.T, v)
u1.shape

torch.Size([3, 1])

In [59]:
u2 = torch.matmul(B, u1)
u2.shape

torch.Size([3, 1])

In [69]:
F22inv = 1. / torch.matmul(v.T, v) - torch.matmul(u1.T, u2)
u3 = F22inv * u2
F11inv = B + F22inv * u2 * u2.T
F11inv.shape, F22inv.shape

(torch.Size([3, 3]), torch.Size([1, 1]))

In [70]:
u3.shape

torch.Size([3, 1])

In [85]:
# Bnew = [F11inv -u3; -u3' F22inv]
Bnew = torch.cat([
    torch.cat([F11inv, -u3], 1),
    torch.cat([-u3.T, F22inv], 1)
], 0)
Bnew

tensor([[ 53.4008,   3.2157, -56.7160,   1.4766],
        [  3.2157,  49.7643, -44.1060,  -5.7923],
        [-56.7160, -44.1060, 110.5267,  -4.4807],
        [  1.4766,  -5.7923,  -4.4807,   8.8822]])

In [86]:
Xnew = torch.cat([X, v], 1)
torch.inverse(torch.matmul(Xnew.T, Xnew))

tensor([[  57.5524,  -13.0699,  -69.3139,   26.4498],
        [ -13.0700,  113.6478,    5.3122, -103.7549],
        [ -69.3140,    5.3122,  148.7549,  -80.2612],
        [  26.4499, -103.7549,  -80.2613,  159.1036]])

tensor([[8.8822]])

In [33]:
u1 = torch.matmul(X.T, v)
u2 = torch.matmul(B, u1)

RuntimeError: size mismatch, [3 x 3], [1] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:631

In [34]:
X.shape, v.shape

(torch.Size([10]), torch.Size([10, 1]))